In [2]:
import tensorflow as tf
tf.__version__

'2.20.0'

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

LABEL_MAP = {
    "mask": 1,
    "no_mask": 0
}

def normalize_label(x):
    if isinstance(x, (int, float, np.integer)):
        return int(x)
    x = str(x).strip().lower()
    if x.isdigit():
        return int(x)
    return LABEL_MAP[x]

In [4]:
df = pd.read_csv("train_labels.csv")
df["target"] = df["target"].apply(normalize_label)

from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df["target"],
    random_state=42
)

In [5]:
IMG_SIZE = 128
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.4),
])

In [8]:
df

,image,target
0,tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg,0
1,rxgismgsvmaayzjarbfjaljhqmpbrt.jpg,1
2,uuzshfrhkgrkolhwdvliqauzulurnz.jpg,0
3,mjspxsagzusaznvnyxgamtrlqkqklp.jpg,0
4,rlbmuajgezfiddjzlyeoupxpqubkpt.jpg,1
...,...,...
1303,hxjwafskxmlfaotwaklzuwuccsbxfu.jpg,0
1304,oyodauphffgmzmvqlykcfvmyxartok.jpg,1
1305,uppvtemhkneqggtbfubeccqjvyefiw.png,1
1306,mzjjvzkzvqmyukzsegtoktaslejcdz.jpg,0


In [9]:
def make_dataset(df, training=True):
    paths = df["image"].apply(lambda x: os.path.join("images/images", x)).values
    labels = df["target"].values

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls=AUTOTUNE)

    if training:
        ds = ds.shuffle(1000)
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)

    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_df, training=True)
val_ds = make_dataset(val_df, training=False)


In [10]:
mixup = tf.keras.layers.MixUp(alpha=0.4)

In [11]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ConvNeXtTiny

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = mixup(inputs)
x = ConvNeXtTiny(
    include_top=False,
    weights="imagenet",
    pooling="avg"
)(x)

x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = models.Model(inputs, outputs)


111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 61s 1us/step


In [13]:
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=3e-4,
    weight_decay=1e-2
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [14]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,              # epochs to wait
    restore_best_weights=True,
    verbose=1
)

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_convnext_mask.keras",  # or .h5
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,   # full model (recommended)
    verbose=1
)

In [16]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [17]:
EPOCHS = 20

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[
        early_stop,
        checkpoint,
        reduce_lr
    ]
    )

Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.4786 - loss: 0.8905 
Epoch 1: val_loss improved from None to 0.73903, saving model to best_convnext_mask.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 436s 11s/step - accuracy: 0.4809 - loss: 0.8297 - val_accuracy: 0.5115 - val_loss: 0.7390 - learning_rate: 3.0000e-04
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.4948 - loss: 0.7518 
Epoch 2: val_loss improved from 0.73903 to 0.69412, saving model to best_convnext_mask.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 409s 11s/step - accuracy: 0.5140 - loss: 0.7363 - val_accuracy: 0.4885 - val_loss: 0.6941 - learning_rate: 3.0000e-04
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.5126 - loss: 0.7325 
Epoch 3: val_loss did not improve from 0.69412
37/37 ━━━━━━━━━━━━━━━━━━━━ 409s 11s/step - accuracy: 0.5021 - loss: 0.7309 - val_accuracy: 0.5115 - val_loss: 0.6968 - learning_rate: 3.0000e-04
Epoch 4/20
28/37 ━━━━━━━━━━━━━━━━━━━━ 1:38 11s/step - accuracy: 0.4984 - loss: 0

KeyboardInterrupt: 